<a href="https://colab.research.google.com/github/sergioberdiales/TFM_KSchool_Gijon_Air_Pollution/blob/master/22_Forecasting_Models_ML_PM10_MULTIVAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we are going to try to improve the forecasts creating multivariate models. 
We will use the XGBoost algorithm and the three years training period (2014-01-01 - 2016-12-31).
Testing period: 2017-01-01 - 2017-09-30. 

### We import the libraries we need to run the algorithms

In [7]:
%pylab inline
import pandas as pd

# We install and import pyreadr, in order to read rds objects.  
# https://github.com/ofajardo/pyreadr

!pip install pyreadr
import pyreadr


# Importing models


import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Importing metrics

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Model selection
from sklearn.model_selection import GridSearchCV

# Variable selection

import sklearn
from sklearn.feature_selection import f_regression


Populating the interactive namespace from numpy and matplotlib


We upload the train and test data. All rds files from this folder  ~\`TFM_KSchool_Gijon_Air_Pollution\train_test\
Forecasting_Models_ML_PM10_MULTIVAR

The train and test datasets were generated running this rmd file "_10_2_train_test_MULTIVAR_datasets.rmd"

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving X_test_PM10_201701_201709_multivar.rds to X_test_PM10_201701_201709_multivar.rds
Saving X_test_PM10_20170101_20170114_multivar.rds to X_test_PM10_20170101_20170114_multivar.rds
Saving X_train_PM10_200901_201612_multivar.rds to X_train_PM10_200901_201612_multivar.rds
Saving X_train_PM10_201401_201612_multivar.rds to X_train_PM10_201401_201612_multivar.rds
Saving X_train_PM10_201610_201612_multivar.rds to X_train_PM10_201610_201612_multivar.rds
Saving X_validation_PM10_201710_201712_multivar.rds to X_validation_PM10_201710_201712_multivar.rds
Saving y_test_PM10_201701_201709_multivar.rds to y_test_PM10_201701_201709_multivar.rds
Saving y_test_PM10_20170101_20170114_multivar.rds to y_test_PM10_20170101_20170114_multivar.rds
Saving y_train_PM10_200901_201612_multivar.rds to y_train_PM10_200901_201612_multivar.rds
Saving y_train_PM10_201401_201612_multivar.rds to y_train_PM10_201401_201612_multivar.rds
Saving y_train_PM10_201610_201612_multivar.rds to y_train_PM10_201610_201612_multi

In [6]:
!ls

sample_data


## Adding the variable 'no_lab_days'
As we saw in the data exploration phase if a day is holiday or not has an effect in the PM10 levels. So, we add the variable 'no_lab_days' to try to improve the model.

#### XGBOOST

In [12]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6631146245341922
R^2 adjusted train: 0.6629191549135831
Mean Absolute Error train: 5.503836677301813
Root Mean Squared Error train: 8.597633475294032
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.5989526095973352
R^2 adjusted test: 0.5980077830140087
Mean Absolute Error test: 4.796642205995463
Root Mean Squared Error test: 6.80743069887127
Standard Deviation test: PM10_0    10.749429
dtype: float64


## Adding the variable 'hour'

The levels of the PM10 pollutant also depends on the hour of the day. We add as a new variable to the model

In [13]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6677327260973054
R^2 adjusted train: 0.6672310061158044
Mean Absolute Error train: 5.450381722721943
Root Mean Squared Error train: 8.538500965207893
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6050019940808449
R^2 adjusted test: 0.6025733448248387
Mean Absolute Error test: 4.7570254454696705
Root Mean Squared Error test: 6.7558940958393645
Standard Deviation test: PM10_0    10.749429
dtype: float64


## Adding the variable 'month'

The levels of the PM10 have a clear monthly seasonality. So, the inclusion of this variable could be some effect in the results.

In [14]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour', 'month']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour', 'month']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6681943593836794
R^2 adjusted train: 0.667680469825285
Mean Absolute Error train: 5.447331970676348
Root Mean Squared Error train: 8.532567449328933
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6051684283849006
R^2 adjusted test: 0.6026781630325506
Mean Absolute Error test: 4.752892953718895
Root Mean Squared Error test: 6.754470631689703
Standard Deviation test: PM10_0    10.749429
dtype: float64


## Adding the week_day variable

In [21]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour', 'month', 'week_day']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 'no_lab_days', 'hour', 'month', 'week_day']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6676343973851251
R^2 adjusted train: 0.6671067512259363
Mean Absolute Error train: 5.44979083115112
Root Mean Squared Error train: 8.539764282067889
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6052312997470766
R^2 adjusted test: 0.6026787817356636
Mean Absolute Error test: 4.751691392509307
Root Mean Squared Error test: 6.753932833161597
Standard Deviation test: PM10_0    10.749429
dtype: float64


##Adding lagged SO2 - NO2 levels as variables
The SO2 and the NO2 are precursors of the PM10 pollutant. So, in theory the levels of these pollutants had to have influence on the PM10 levels. We include the one hour lagged levels of these pollutants to check if they increase the explanatory power of the model. 

In [22]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                   'no_lab_days', 'hour', 'month', 'week_day','NO2_1', 'SO2_1']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                 'no_lab_days', 'hour', 'month', 'week_day','NO2_1', 'SO2_1']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6722011798746166
R^2 adjusted train: 0.671655356250647
Mean Absolute Error train: 5.411945677774943
Root Mean Squared Error train: 8.480892156395758
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6015486568153066
R^2 adjusted test: 0.5988458002516621
Mean Absolute Error test: 4.814010842149647
Root Mean Squared Error test: 6.785362105465956
Standard Deviation test: PM10_0    10.749429
dtype: float64


 The MAE increases. We remove these variables.



#### We add the two hour lagged NO2-SO2 variables

In [25]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]


# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                   'no_lab_days', 'hour', 'month', 'week_day','NO2_2', 'SO2_2']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                 'no_lab_days', 'hour', 'month', 'week_day', 'NO2_2', 'SO2_2']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6699616863321667
R^2 adjusted train: 0.6694121336878158
Mean Absolute Error train: 5.4299401094573865
Root Mean Squared Error train: 8.509813209374219
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6051238277725847
R^2 adjusted test: 0.6024452230390653
Mean Absolute Error test: 4.764689330408576
Root Mean Squared Error test: 6.7548521171643925
Standard Deviation test: PM10_0    10.749429
dtype: float64


The  MAE increases again. We remove these variables from the model.

# Weather variables

##Adding lagged vv - wd (wind speed and wind direction)

In [27]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120',
                   'PM10_144', 'PM10_168', 'no_lab_days', 'hour','month', 'week_day', 'vv_1','wd_1']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 
                 'PM10_144', 'PM10_168', 'no_lab_days', 'hour','month', 'week_day', 'vv_1','wd_1']]

X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6735418428037112
R^2 adjusted train: 0.6728082005425858
Mean Absolute Error train: 5.398101778568063
Root Mean Squared Error train: 8.463531405835221
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6129310378324916
R^2 adjusted test: 0.6093810694887668
Mean Absolute Error test: 4.745920449606892
Root Mean Squared Error test: 6.687742691925128
Standard Deviation test: PM10_0    10.749429
dtype: float64


## Adding rainfall (LL)

In [28]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'month', 'week_day', 'vv_1', 'wd_1', 'LL_1', 'LL_2']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                 'no_lab_days', 'hour', 'month', 'week_day', 'vv_1', 'wd_1', 'LL_1', 'LL_2']]

X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6750761873495172
R^2 adjusted train: 0.6743207555380308
Mean Absolute Error train: 5.385328554318593
Root Mean Squared Error train: 8.443618796273906
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6137750539111198
R^2 adjusted test: 0.6101095213636138
Mean Absolute Error test: 4.741300497122194
Root Mean Squared Error test: 6.68044729587217
Standard Deviation test: PM10_0    10.749429
dtype: float64


Adding the LL variable (rainfall) barely improves the model. 

## Adding solar radiation (RS)

In [30]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120','PM10_144', 'PM10_168',
                   'month', 'week_day', 'no_lab_days', 'hour',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120','PM10_144', 'PM10_168',
                   'month', 'week_day', 'no_lab_days', 'hour',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2']]

X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6746801275947865
R^2 adjusted train: 0.6738985026349289
Mean Absolute Error train: 5.383231667215004
Root Mean Squared Error train: 8.448763324005329
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6149008632001653
R^2 adjusted test: 0.6111229919214328
Mean Absolute Error test: 4.723713980252608
Root Mean Squared Error test: 6.670703753430319
Standard Deviation test: PM10_0    10.749429
dtype: float64


##Adding two hours lagged NO2 and SO2 again
We recover these variables in order to see if 
first one lag

In [33]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'month', 'week_day',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2','NO2_1', 'SO2_1']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'month', 'week_day',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2','NO2_1', 'SO2_1']]

X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6780333930577379
R^2 adjusted train: 0.6772348090619117
Mean Absolute Error train: 5.36435468197966
Root Mean Squared Error train: 8.405107320564705
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6108817120272232
R^2 adjusted test: 0.6069400262991039
Mean Absolute Error test: 4.759544507377988
Root Mean Squared Error test: 6.705423353788262
Standard Deviation test: PM10_0    10.749429
dtype: float64


##Adding two hours lagged NO2 and SO2 again
We recover these variables in order to see if 
two hours lagged

In [34]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'month', 'week_day',
                    'no_lab_days', 'hour','PM10_144', 'PM10_168',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2','NO2_2', 'SO2_2']]

X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'month', 'week_day',
                    'no_lab_days', 'hour','PM10_144', 'PM10_168',
                   'vv_1', 'wd_1', 'LL_1', 'LL_2', 'RS_1', 'RS_2','NO2_2', 'SO2_2']]

X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.6771478564985056
R^2 adjusted train: 0.6763470760782406
Mean Absolute Error train: 5.358657957088558
Root Mean Squared Error train: 8.416658082130033
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6174061016366456
R^2 adjusted test: 0.613530506591496
Mean Absolute Error test: 4.706147752618614
Root Mean Squared Error test: 6.64897042576772
Standard Deviation test: PM10_0    10.749429
dtype: float64


### All the variables
### Gridsearchcv  
 "n_estimators":[100, 500, 1000, 2000],  
                                "min_samples_leaf":[10,30],  
                                  "max_depth":range(2,5)},  
                                    scoring="neg_mean_absolute_error")  


In [0]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)

X_train = X_train[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'NO2_1', 'SO2_1', 'NO2_2', 'SO2_2', 'vv_1', 
                   'wd_1', 'LL_1', 'LL_2', 'week_day', 'month']]
X_test = X_test[['PM10_1', 'PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                 'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                 'no_lab_days', 'hour', 'NO2_1', 'SO2_1', 'NO2_2', 'SO2_2', 'vv_1', 
                 'wd_1', 'LL_1', 'LL_2', 'week_day', 'month']]




X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = GridSearchCV(XGBRegressor(n_estimators=350, min_samples_leaf=1,max_depth=4, random_state=42),
                   param_grid={"n_estimators":[100, 500, 1000, 2000],
                                "min_samples_leaf":[10,30],
                                  "max_depth":range(2,5)},
                                    scoring="neg_mean_absolute_error")

regXGB.fit(X_train,y_train)

print(regXGB.best_params_)
print("Best score: {}".format(regXGB.best_score_))


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 2, 'min_samples_leaf': 10, 'n_estimators': 500}
Best score: -5.601586251842673
R^2 train: 0.6827653548537419
R^2 adjusted train: 0.6817688448771326
Mean Absolute Error train: 5.311897631850796
Root Mean Squared Error train: 8.343113522235745
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6183819125339685
R^2 adjusted test: 0.61347617295537
Mean Absolute Error test: 4.742529574133801
Root Mean Squared Error test: 6.640485867950488
Standard Deviation test: PM10_0    10.749429
dtype: float64


{'max_depth': 2, 'min_samples_leaf': 10, 'n_estimators': 500}
Best score: -5.601586251842673
R^2 train: 0.6827653548537419
R^2 adjusted train: 0.6817688448771326
Mean Absolute Error train: 5.311897631850796
Root Mean Squared Error train: 8.343113522235745
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.6183819125339685
R^2 adjusted test: 0.61347617295537
Mean Absolute Error test: 4.742529574133801
Root Mean Squared Error test: 6.640485867950488
Standard Deviation test: PM10_0    10.749429
dtype: float64

# PM10 forecasts 2 hours ahead
n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42

In [36]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)

X_train = X_train[['PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'month', 'week_day','no_lab_days', 'hour',
                    'PM10_144', 'PM10_168', 'vv_2', 'wd_2', 'LL_2', 'RS_2','NO2_2', 'SO2_2']]
                   
X_test = X_test[['PM10_2', 'PM10_3', 'PM10_4', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'month', 'week_day','no_lab_days', 'hour',
                    'PM10_144', 'PM10_168', 'vv_2', 'wd_2', 'LL_2', 'RS_2','NO2_2', 'SO2_2']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.5702202383228376
R^2 adjusted train: 0.5689203499436519
Mean Absolute Error train: 6.336664454894228
Root Mean Squared Error train: 9.710930100285669
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.47676922203065597
R^2 adjusted test: 0.4702952371509591
Mean Absolute Error test: 5.5608391185893336
Root Mean Squared Error test: 7.775566190657504
Standard Deviation test: PM10_0    10.749429
dtype: float64


# PM10 forecasts 6 hours ahead
n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42

In [37]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)


X_train = X_train[['PM10_6','PM10_7', 'PM10_8', 'PM10_9', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'month', 'week_day','no_lab_days', 'hour']]

X_test = X_test[['PM10_6','PM10_7', 'PM10_8', 'PM10_9', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'month', 'week_day','no_lab_days', 'hour']]



X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.4016048048015286
R^2 adjusted train: 0.4002600443954101
Mean Absolute Error train: 7.821355014675849
Root Mean Squared Error train: 11.458608900573376
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.2680946370432574
R^2 adjusted test: 0.261382032512661
Mean Absolute Error test: 6.763604330901764
Root Mean Squared Error test: 9.196293983085365
Standard Deviation test: PM10_0    10.749429
dtype: float64


# PM10 forecasts 6 hours ahead
### Gridsearchcv  
 "n_estimators":[100, 500, 1000, 2000],  
                                "min_samples_leaf":[10,30],  
                                  "max_depth":range(2,5)},  
                                    scoring="neg_mean_absolute_error")  


In [0]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)

X_train = X_train[['PM10_6','PM10_7', 'PM10_8', 'PM10_9', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'month', 'week_day','no_lab_days', 'hour']]

X_test = X_test[['PM10_6','PM10_7', 'PM10_8', 'PM10_9', 'PM10_22', 'PM10_23', 'PM10_24', 
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'month', 'week_day','no_lab_days', 'hour']]


X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = GridSearchCV(XGBRegressor(n_estimators=350, min_samples_leaf=1,max_depth=4, random_state=42),
                   param_grid={"n_estimators":[100, 500, 1000, 2000],
                                "min_samples_leaf":[10,30],
                                  "max_depth":range(2,5)},
                                    scoring="neg_mean_absolute_error")

regXGB.fit(X_train,y_train)

print(regXGB.best_params_)
print("Best score: {}".format(regXGB.best_score_))


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


# PM10 forecasts 12 hours ahead

In [18]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)

X_train = X_train[['PM10_12','PM10_13', 'PM10_14', 'PM10_15', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'week_day', 'month']]

X_test = X_test[['PM10_12','PM10_13', 'PM10_14', 'PM10_15', 'PM10_22', 'PM10_23', 'PM10_24',
                   'PM10_48', 'PM10_72', 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168', 
                   'no_lab_days', 'hour', 'week_day', 'month']]



X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.32493202522625597
R^2 adjusted train: 0.3234149597631665
Mean Absolute Error train: 8.389725768295612
Root Mean Squared Error train: 12.170589109352777
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.16328901405336615
R^2 adjusted test: 0.15561519413165448
Mean Absolute Error test: 7.379480929525691
Root Mean Squared Error test: 9.832707431450373
Standard Deviation test: PM10_0    10.749429
dtype: float64


# PM10 forecasts 24 hours ahead

In [19]:
X_train = pyreadr.read_r("X_train_PM10_201401_201612_multivar.rds")
y_train = pyreadr.read_r("y_train_PM10_201401_201612_multivar.rds")

X_test = pyreadr.read_r("X_test_PM10_201701_201709_multivar.rds")
y_test = pyreadr.read_r("y_test_PM10_201701_201709_multivar.rds")

X_train = X_train[None]
y_train = y_train[None]
X_test = X_test[None]
y_test = y_test[None]

# We convert the 'hour' variable to string
X_train.hour = X_train.hour.astype(str)
X_test.hour = X_test.hour.astype(str)

# We convert the 'week_day' variable to string
X_train.week_day = X_train.week_day.astype(str)
X_test.week_day = X_test.week_day.astype(str)

# We convert the 'month' variable to string
X_train.month = X_train.month.astype(str)
X_test.month = X_test.month.astype(str)

X_train = X_train[['PM10_24','PM10_25', 'PM10_26', 'PM10_27', 'PM10_48', 'PM10_72',
                   'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                   'no_lab_days', 'hour','week_day', 'month']]

X_test = X_test[['PM10_24','PM10_25', 'PM10_26', 'PM10_27', 'PM10_48', 'PM10_72',
                 'PM10_96', 'PM10_120', 'PM10_144', 'PM10_168',
                 'no_lab_days', 'hour', 'week_day', 'month']]



X_train = pd.get_dummies(X_train)
X_test  = pd.get_dummies(X_test)


# Al hacer el one hot encoding el X_train y el X_test tienen distintas dimensiones. X_train tiene 47 variables y X_test 44. Por que? Porque el dataset de test tiene solo 9 meses, de enero a septiembre, y entonces la variable month solo se convierte en 9 variables, mientras que el dataset tendriamos 12, los 12 meses del anho. 
# Con el siguiente codigo solucionamos el problema (falta referencia)

# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
# This code also ensure that column resulting from category in the test dataset but not present in the training dataset will be removed



regXGB = XGBRegressor(n_estimators=500, min_samples_leaf=10,max_depth=2, random_state=42)
regXGB.fit(X_train,y_train)


# Compute train scores

y_pred = regXGB.predict(X_train)

r2_train = r2_score(y_train, y_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))
mae_train = mean_absolute_error(y_train, y_pred)
r2_adjusted_train = 1 - (1-r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
sd_train = std(y_train)

print("R^2 train: {}".format(r2_train))
print("R^2 adjusted train: {}".format(r2_adjusted_train))
print("Mean Absolute Error train: {}".format(mae_train))
print("Root Mean Squared Error train: {}".format(rmse_train)) 
print("Standard Deviation train: {}".format(sd_train))  

# Compute test scores

y_pred = regXGB.predict(X_test)

r2_test = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_adjusted_test = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
sd_test = std(y_test)

print("R^2 test: {}".format(r2_test))
print("R^2 adjusted test: {}".format(r2_adjusted_test))
print("Mean Absolute Error test: {}".format(mae_test))
print("Root Mean Squared Error test: {}".format(rmse_test))
print("Standard Deviation test: {}".format(sd_test))  


R^2 train: 0.28464814701162733
R^2 adjusted train: 0.2831238810921185
Mean Absolute Error train: 8.712622302391988
Root Mean Squared Error train: 12.528460294007024
Standard Deviation train: PM10_0    14.812823
dtype: float64
R^2 test: 0.08858207273456675
R^2 adjusted test: 0.08065920470870958
Mean Absolute Error test: 7.827284202333162
Root Mean Squared Error test: 10.262287204288324
Standard Deviation test: PM10_0    10.749429
dtype: float64
